In [ ]:
# ------------------------------------------------------
#          1. Linear Congruential Generator (LCG)
# ------------------------------------------------------
def lcg(seed = 123):
    """
    Simple LCG using Park-Miller values.
    Each call updates seed and returns a U(0,1) random number.
    """
    a = 16807
    m = 2**31 - 1
    seed = (a * seed) % m
    return seed, seed / m


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from basic_operations import mean, var, std

In [ ]:
# Initializaion
a,b = 0,np.pi # limits
N = int(1e3) # no. of samples
seed = 123 
xrand = np.zeros(N) # array of samples

for i in range(len(xrand)):
    seed, xrand[i] =  lcg(seed)
    xrand[i]  = a + (b-a) * xrand[i]

In [ ]:
# function to integrate
def func(x):
    return np.sin(x)

In [ ]:
integral = 0
for i in range(N):
    integral += func(xrand[i])

answer = (b-a)/float(N) * integral
print(answer)

----

In [ ]:
def func(x):
    return 4 / (1 + x**2)

a,b = 0,1 # limits of integration
integral = 0
N = int(1e3) # no of sampled points

xrand = np.zeros(N) # stores sampled points
f_values = np.zeros(N) # stores f(x) for each point

# populating xrand
for i in range(len(xrand)):
    seed, xrand[i] =  lcg(seed)
    xrand[i]  = a + (b-a) * xrand[i]

# populating f_values
for i in range(N):
    f_values[i] = func(xrand[i])


integral = np.mean(f_values) * (b-a)
print("Integral: ", integral)

In [ ]:
standard_error = np.sqrt(np.var(f_values))/np.sqrt(N)
confidence_interval = [integral - 2.576 * standard_error, integral + 2.576 * standard_error]
abs_error = np.abs(integral - np.pi)
print("Standard error: ", standard_error)
print("confidence_interval: ", confidence_interval)
print("absolute error: ", abs_error)

### Evaluating for different values of N

In [ ]:
a,b = 0, 1
integral = 0
integral_vals = []
N_vals = [1e3, 1e4, 1e5,1e6]
margins = []
se_squared = []
variances = []


for i in N_vals:
    N = int(i)

    xrand = np.zeros(N)
    f_values = np.zeros(N)

    for i in range(len(xrand)):
        seed, xrand[i] =  lcg(seed)
        xrand[i]  = a + (b-a) * xrand[i]

    for i in range(N):
        f_values[i] = func(xrand[i])


    integral = np.mean(f_values) * (b-a)
    standard_error = np.sqrt(np.var(f_values))/np.sqrt(N) * (b-a)
    confidence_interval = [integral - 2.576 * standard_error, integral + 2.576 * standard_error]
    abs_error = np.abs(integral - np.pi)
    integral_vals.append(integral)
    margins.append(2.576 * standard_error)
    variances.append(np.var((f_values)))
    se_squared.append(standard_error**2)



In [ ]:
# Plot the error bars
# fmt='-o' connects points with a line and uses circle markers
# capsize=5 adds the horizontal lines at the top/bottom of the error bars
plt.errorbar(N_vals, integral_vals, yerr=margins, fmt='-o', 
             capsize=5, label=r'$\hat{I} \pm 99\% CI$')

# Plot the True Pi reference line
plt.axhline(y=np.pi, color='r', linestyle='--', label=r'True $\pi$')

# 5. Formatting to match the screenshot
plt.xscale('log') # Logarithmic X-axis for 10^3, 10^4...
plt.xlabel(r'Number of samples $N$')
plt.ylabel(r'Estimate $\hat{I}$')
plt.title('Convergence of Monte Carlo Estimate')
plt.grid(True, which="both", linestyle=':', alpha=0.7) # Dotted grid lines
plt.legend()

plt.show()

In [ ]:
# 4. Plotting
plt.figure(figsize=(8, 6))

# Use loglog for both axes as shown in the screenshot
plt.loglog(N_vals, variances, 'o-', label=r'Var($f$)')
plt.loglog(N_vals, se_squared, 's--', label=r'SE$^2 = \mathrm{Var}(f)/N$')

# 5. Formatting
plt.xlabel(r'Number of samples $N$')
plt.ylabel(r'Variance / SE$^2$')
plt.title('Scaling of Variance and Standard Error')
plt.grid(True, which="both", linestyle=':', alpha=0.7)
plt.legend()

plt.show()

----

## 6-d Integration

In [ ]:
import numpy as np

# --- 1. LCG Function ---
def lcg(seed):
    a = 16807
    m = 2**31 - 1
    seed = (a * seed) % m
    return seed, seed / m

# --- 2. The 6D Function g(x) ---
def g(x):
    # x is an array of size 6
    # Term 1: Sum of squares x_k^2 for k=0 to 5
    term1 = sum(x**2)
    
    # Term 2: Sum of (x_k - x_{k+3})^2 for k=0 to 2
    term2 = (x[0] - x[3])**2 + (x[1] - x[4])**2 + (x[2] - x[5])**2
    
    return np.exp(-term1 - 0.5 * term2)

# --- 3. Setup ---
a_lim, b_lim = -5, 5 # limits of integral
d = 6 # dimensions
vol = (b_lim - a_lim)**d  # Volume of the 6D hypercube (10^6)
true_val = 10.966         # Given in problem
seed = 123456

N_vals = [1e4, 1e5, 1e6]

print(f"{'N':<9} {'Estimate':<10} {'SE':<10} {'95% CI':<20} {'Rel Error':<10} {'R (Ratio)'}")
print("-" * 85)

for n_val in N_vals:
    N = int(n_val)
    
    # Pre-allocate
    g_values = np.zeros(N)
    
    # --- Generation & Evaluation Loop ---
    for i in range(N):
        # Generate 6 random numbers
        x_point = np.zeros(d)
        for j in range(d):
            seed, u = lcg(seed)
            # SCALE CORRECTLY: a + (b-a)*u
            x_point[j] = a_lim + (b_lim - a_lim) * u
            
        g_values[i] = g(x_point)

    # --- Statistics ---
    # Mean of g
    mean_g = mean(g_values)
    
    # Integral Estimate = Volume * Mean
    I_hat = vol * mean_g
    
    # Variance of g (sample variance)
    var_g = var(g_values)
    
    # Standard Error = Volume * sqrt(Var(g) / N)
    SE = vol * np.sqrt(var_g / N)
    
    # 95% CI (Z = 1.96)
    z_score = 1.96
    ci_lower = I_hat - z_score * SE
    ci_upper = I_hat + z_score * SE
    
    # Relative Error
    rel_error = np.abs(I_hat - true_val) / true_val

    # --- Task 2: Efficiency Ratio R ---
    # E[g^2] approx mean(g^2)
    E_g2 = np.mean(g_values**2)
    # E[g]^2 approx mean(g)^2
    E_g_sq = mean_g**2
    
    # R = E[g^2] / E[g]^2
    # Note: If mean_g is very close to 0, this might be unstable, 
    # but for this integral it should be fine.
    R = E_g2 / E_g_sq

    # Output Row
    print(f"{int(N):<9} {I_hat:<10.4f} {SE:<10.4f} [{ci_lower:.3f}, {ci_upper:.3f}]   {rel_error:<10.4f} {R:.2f}")

----

## n dimensional Sphere Volume 

In [ ]:
import math

# True value of volume according to formula specified above
def analytic_sphere_volume(n):
    """Calculates the true volume of a unit n-sphere."""
    return (math.pi**(n/2)) / math.gamma(n/2 + 1)

In [ ]:
def g(x):
    r_2 = sum(x**2)
    return 1 if r_2 <= 1 else 0

In [ ]:
# --- 2. Setup ---
a_lim, b_lim = 0, 1
d = 2 

# Volume of the integration domain (Hypercube [0,1]^6)
vol_hypercube = (b_lim - a_lim)**d 

# Calculate the TRUE volume for comparison
true_val_full = analytic_sphere_volume(d)

# NOTE: The integral over [0,1] is only 1/(2^n) of the full sphere.
true_val_region = true_val_full / (2**d)

seed = 123456
rng = np.random.default_rng(seed) 

N_vals = [1e4, 1e5, 1e6]

print(f"Dimension: {d}")
print(f"True Volume (Full Sphere): {true_val_full:.5f}")
print(f"True Integral (Region [0,1]^{d}): {true_val_region:.5f}")
print("-" * 100)
print(f"{'N':<9} {'Estimate':<10} {'SE':<10} {'95% CI':<25} {'Rel Error %':<12}")
print("-" * 100)

for n_val in N_vals:
    N = int(n_val)
    
    # Pre-allocate array for g(x) results
    g_values = np.zeros(N)
    
    # --- Generation & Evaluation Loop ---
    for i in range(N):
        x_point = rng.uniform(a_lim, b_lim, d)
        g_values[i] = g(x_point)

    # --- Statistics ---
    mean_g = np.mean(g_values)
    
    # Integral Estimate
    I_hat = vol_hypercube * mean_g * (2**d) # 2^d to get total volume 
    
    # Standard Error
    variance = mean_g * (1 - mean_g) 
    se_g = np.sqrt(variance / N)
    
    # Scale SE by volume of hypercube
    se_volume = vol_hypercube * se_g
    
    # 95% Confidence Interval
    ci_lower = I_hat - (1.96 * se_volume)
    ci_upper = I_hat + (1.96 * se_volume)
    
    # Relative Error
    rel_error = abs(I_hat - true_val_region) / true_val_region

    print(f"{N:<9.0f} {I_hat:<10.5f} {se_volume:<10.5f} [{ci_lower:.5f}, {ci_upper:.5f}]   {rel_error*100:<10.4f}")

----

# Importance sampling

In [ ]:
# --- 2. Manual Helper Functions (No scipy allowed) ---

def f(x):
    """The function to integrate: e^(-2|x-5|)"""
    # np.exp and np.abs are basic element-wise math (allowed)
    return np.exp(-2 * np.abs(x - 5))

# --- 2. Probability Functions ---
def q_pdf_gaussian(x, mu, sigma):
    """
    The Proposal Distribution: N(5, 1)
    """
    coeff = 1.0 / (sigma * math.sqrt(2 * math.pi))
    exponent = -0.5 * ((x - mu) / sigma)**2
    return coeff * np.exp(exponent)

def p_val_uniform(x):
    """
    The Target P(x). 
    NOTE: To match the slide's return statement (which has no *10 multiplier),
    p(x) here acts as the 'Indicator Function' (1.0) inside the bounds,
    rather than the PDF density (0.1). This allows the 'mean' to equal the 'Integral'.
    """
    if 0 <= x <= 10:
        return 1.0 
    return 0.0

# --- 3. The Slide Implementation ---
def run_simulation(N=10000):
    
    # ===============================================
    # STRATEGY 1: Crude Monte Carlo (Uniform)
    # Logic: Integral = Volume * Mean(f)
    # ===============================================
    
    # 1. Sample Uniformly [0, 10]
    x_crude = np.random.uniform(0, 10, N)
    
    # 2. Calculate f_i
    # Using list comprehension since we aren't using vector operations for logic
    f_vals_crude = [np.exp(-2 * np.abs(val - 5)) for val in x_crude]
    
    # 3. Slide Logic Return
    # I = 10 * mean(f_i)
    I_crude = 10 * mean(f_vals_crude)
    
    # Var = (10^2 / N) * var(f_i)
    Var_crude = (10**2 / N) * var(f_vals_crude)


    # ===============================================
    # STRATEGY 2: Importance Sampling (Gaussian)
    # Logic: Integral = Mean(f * w)
    # ===============================================
    
    weighted_terms = []
    
    # 1. Sample Normal(5, 1)
    x_is = np.random.normal(5, 1, N)
    
    for val in x_is:
        # Check bounds (Slide logic: if 0 <= x_i <= 10)
        if 0 <= val <= 10:
            
            # Calculate q(x) - The Gaussian probability
            q_i = q_pdf_gaussian(val, 5, 1)
            
            # Calculate p(x)
            p_i = p_val_uniform(val) 
            
            # w_i = p(x_i) / q(x_i)
            w_i = p_i / q_i
            
            # f_i = exp(-2 * |x - 5|)
            f_i = f(val)
            
            # store f_i * w_i
            weighted_terms.append(f_i * w_i)
            
        else:
            # If outside bounds, the contribution to the integral is 0
            weighted_terms.append(0.0)
            
    # 3. Slide Logic Return
    # Note: No '10' multiplier here, exactly as shown in the slide.
    I_is = mean(weighted_terms)
    
    # Var = var(f_i * w_i) / N
    # Note: No '100' multiplier here, exactly as shown in the slide.
    Var_is = var(weighted_terms) / N

    return I_crude, Var_crude, I_is, Var_is



# --- Run and Print Results ---
if __name__ == "__main__":
    N_samples = [1e2, 1e3,1e4,1e5]
    Ic_vals = []
    Ii_vals = []
    Vc_vals = []
    Vi_vals = []
    for i in N_samples:
        I_c, V_c, I_i, V_i = run_simulation(int(i))
        Ic_vals.append(I_c)
        Ii_vals.append(I_i)
        Vc_vals.append(V_c)
        Vi_vals.append(V_i)

    print(f"--- Results for N={N_samples} ---")
    print(f"True Integral Value: ~1.000 (approx)")
    print("-" * 30)
    print(f"Crude Monte Carlo:")
    print(f"  Estimate: {I_c:.5f}")
    print(f"  Variance: {V_c:.6f}")
    print("-" * 30)
    print(f"Importance Sampling:")
    print(f"  Estimate: {I_i:.5f}")
    print(f"  Variance: {V_i:.6f}")
    print("-" * 30)
    
    # Avoid division by zero if variance is extremely small
    if V_i > 0:
        ratio = V_c / V_i
        print(f"Variance Reduction Factor: {ratio:.1f}x")
    else:
        print("Variance Reduction: Infinite (perfect match)")

In [ ]:
# Plot the error bars
# fmt='-o' connects points with a line and uses circle markers
# capsize=5 adds the horizontal lines at the top/bottom of the error bars
plt.plot(N_samples, Ic_vals, label=r'I crude')
plt.plot(N_samples, Ii_vals, label = r'I MC')
# Plot the True Pi reference line
plt.axhline(y=1, color='r', linestyle='--', label=r'True value')

# 5. Formatting to match the screenshot
plt.xscale('log') # Logarithmic X-axis for 10^3, 10^4...
plt.xlabel(r'Number of samples $N$')
plt.ylabel(r'Estimate $\hat{I}$')
plt.title('Convergence of Monte Carlo Estimate')
plt.grid(True, which="both", linestyle=':', alpha=0.7) # Dotted grid lines
plt.legend()

plt.show()

In [ ]:
# Plot the error bars
# fmt='-o' connects points with a line and uses circle markers
# capsize=5 adds the horizontal lines at the top/bottom of the error bars
plt.plot(N_samples, Vc_vals, label=r'I=V crude')
plt.plot(N_samples, Vi_vals, label = r'V MC')

# 5. Formatting to match the screenshot
plt.xscale('log') # Logarithmic X-axis for 10^3, 10^4...
plt.xlabel(r'Number of samples $N$')
plt.ylabel(r'Var$')
plt.title('Variance vs Sample size')
plt.grid(True, which="both", linestyle=':', alpha=0.7) # Dotted grid lines
plt.legend()

plt.show()